In [9]:
import os
os.chdir(r'C:\Users\user\Python Programs\Resume Projects\End-to-end-Wine-quality')
print(os.getcwd())

C:\Users\user\Python Programs\Resume Projects\End-to-end-Wine-quality


In [10]:
from dataclasses import dataclass
from pathlib import Path

In [11]:
@dataclass(frozen=True)
class ModelTrainerConfig():
    root_dir:Path
    train_data_path: Path
    test_data_path: Path
    model_name:str
    alpha: float
    l1_ratio: float
    target_column: str

In [12]:
from mlProject.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH
from mlProject.utils.common import read_yaml,crate_directories

In [13]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH
                 ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        crate_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)->ModelTrainerConfig:
        config=self.config.model_trianer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        crate_directories([config.root_dir])
        model_trainer_config=ModelTrainerConfig(root_dir=config.root_dir,train_data_path=config.train_data_path,test_data_path=config.test_data_path,
                                                model_name=config.model_name,alpha=params.alpha,l1_ratio=params.l1_ratio,target_column=schema.name)
        return model_trainer_config

In [14]:
import pandas as pd
from mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [15]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config=config
        
    def train(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)

        train_x=train_data.drop([self.config.target_column],axis=1)
        train_y=train_data[[self.config.target_column]]
        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data[[self.config.target_column]]

        lr=ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        lr.fit(train_x,train_y)
        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))

In [16]:
try:
    config=ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer=ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-06-21 18:46:50,198: INFO: common: yaml file:config\config.yaml loaded sucessfully]
[2025-06-21 18:46:50,199: INFO: common: yaml file:params.yaml loaded sucessfully]
[2025-06-21 18:46:50,201: INFO: common: yaml file:schema.yaml loaded sucessfully]
[2025-06-21 18:46:50,201: INFO: common: created directory at:artifacts]
[2025-06-21 18:46:50,202: INFO: common: created directory at:artifacts/model_trianer]
